In [20]:
import os, sys
import ee
import geemap
import geopandas as gpd

project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path:
    sys.path.insert(0, project_root)
from Code.utils.utility import load_config, resolve_path

# Set up
config = load_config()
roi_gdf = gpd.read_file(resolve_path(config["SSA_Arid_by_Country_shp_path"]))
ee.Authenticate(scopes=['https://www.googleapis.com/auth/drive'])
ee.Initialize()

# Convert ROI to Earth Engine FeatureCollection
roi_fc = geemap.geopandas_to_ee(roi_gdf)
roi_geom = roi_fc.geometry()

# Define Sentinel-2 RGB Composite
start_date = '2015-06-23'
end_date = '2015-12-31'
rgb_bands = ['B4', 'B3', 'B2']  # Red, Green, Blue bands

# Tile grid setup
tile_size = 10
scale = 10
folder = 'CPIS_Tiles_2015'

minx, miny, maxx, maxy = roi_gdf.total_bounds
lons = list(round(minx + i * tile_size, 6) for i in range(int((maxx - minx) / tile_size)))
lats = list(round(miny + i * tile_size, 6) for i in range(int((maxy - miny) / tile_size)))

task_count = 0
for lon in lons:
    for lat in lats:
        tile_coords = [lon, lat, lon + tile_size, lat + tile_size]
        tile_geom = ee.Geometry.Rectangle(tile_coords)

        s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
            .filterDate(start_date, end_date) \
            .filterBounds(tile_geom) \
            .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
            .select(rgb_bands)
        
        # Composite and scale
        rgb = s2.median().multiply(255).clamp(0, 255).toUint8()

        # Export image
        task = ee.batch.Export.image.toDrive(
            image = rgb,
            description = f'S2_RGB_2015_{lon}_{lat}',
            folder = folder,
            fileNamePrefix = f's2_2015_rgb_{lon}_{lat}',
            region = tile_coords,
            scale = scale,
            crs = 'EPSG:4326',
            maxPixels = 1e13
        )

        task.start()
        print(f"Started export for tile {lon}, {lat}")
        task_count += 1

print(f"Started {task_count} export tasks.")

Started export for tile -17.541667, -34.833333
Started export for tile -17.541667, -24.833333
Started export for tile -17.541667, -24.833333
Started export for tile -17.541667, -14.833333
Started export for tile -17.541667, -14.833333
Started export for tile -17.541667, -4.833333
Started export for tile -17.541667, -4.833333
Started export for tile -17.541667, 5.166667
Started export for tile -17.541667, 5.166667
Started export for tile -17.541667, 15.166667
Started export for tile -17.541667, 15.166667
Started export for tile -7.541667, -34.833333
Started export for tile -7.541667, -34.833333
Started export for tile -7.541667, -24.833333
Started export for tile -7.541667, -24.833333
Started export for tile -7.541667, -14.833333
Started export for tile -7.541667, -14.833333
Started export for tile -7.541667, -4.833333
Started export for tile -7.541667, -4.833333
Started export for tile -7.541667, 5.166667
Started export for tile -7.541667, 5.166667
Started export for tile -7.541667, 15